In [1]:
import numpy as np

## Dive in math
---
from norm formula

$$ LN(x) = \frac{x - \mu}{\sigma + \epsilon} \gamma + \beta$$

from the cross-entropy and softmax gradient we got

$$dZ = \frac {\partial \mathcal {L}} {\partial Z} = \frac {\partial \mathcal {L}}{\partial P} \frac {\partial P} {\partial Z} = \frac{P - Y} {N}$$

where 
* $Z$ is logits
* $P$ is $softmax\left(Z\right)$
* $N$ is token count in loss after mask
* $\mathcal {L}$ is loss function cross entropy

at the LM Head we got

$$Z = h\ @\ W_{vocab} + b_{vocab}$$

where 
* $h \in \mathbb{R} ^ {B \times T \times d_{model}}$
* $W \in \mathbb{R} ^ {d_model \times V}$
* $b \in \mathbb{R} ^ {V}$

$$\frac {\partial \mathcal {L}}{\partial W} = dW = \sum_{b, t} h_{b,t}^\intercal dZ_{b, t}$$

$$\frac {\partial \mathcal {L}}{\partial b} = db = \sum_{b, t} dZ_{b, t}$$

and $dh$

$$ \frac {\partial \mathcal {L}} {\partial h} = \frac {\partial \mathcal {L}} {\partial Z} \cdot \frac {\partial Z} {\partial h} = dh$$

$$dh_{b, t} = dZ_{b, t} W^\intercal$$

We will start from here ->

$$h = LN(y)$$
when $y$ is an output matrix from transformer stack



$$\frac {\partial LN} {\partial \gamma} = `$$